In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
from ModulesPath.plotUtil import Fig
import subjects

sess = subjects.Sd().ratSday3[0]

In [ ]:
spikes_folder = "/data/Clustering/SleepDeprivation/RatN/Day1/spykcirc/clus_combined"
sess.neurons.from_Phy(spikes_folder)

In [ ]:
file = "/data/Clustering/SleepDeprivation/RatN/Day1/RatN_Day1_2019-10-09_03-52-32.test_position.npy"

data = np.load(file, allow_pickle=True).item()

data

In [ ]:
import numpy as np
import pandas as pd
from ModulesPath.core import Epoch

a = pd.DataFrame({"start": [1, 2, 3], "stop": [3, 4, 5], "label": [3, 4, 6]})

b = np.where(a["label"] == 3)[0]

a.loc[0][["start", "stop"]].values

d = pd.DataFrame(columns=["start", "stop"])

d.empty


## Convert paradigm epochs to current format

In [ ]:
import pandas as pd
import numpy as np

file = "/data/Clustering/SleepDeprivation/RatN/Day2/RatN_Day2_2019-10-11_03-58-54_epochs.npy"

data = np.load(file, allow_pickle=True).item()

epochs = pd.DataFrame(data)
a = epochs.T.reset_index()

epochs_new = pd.DataFrame(
    {"start": a[0], "stop": a[1], "label": a["index"].str.lower()}
)
metadata = None

data = {"epochs": epochs_new, "metadata": metadata}

# np.save(file[:-11] + ".paradigm.npy", data)


In [ ]:
epochs_new

## Convert artifact to current format

In [ ]:
import pandas as pd
import numpy as np

file = "/data/Clustering/SleepDeprivation/RatS/Day3SD/RatS_Day3SD_2020-11-29_07-53-30.artifact.npy"

data = np.load(file, allow_pickle=True).item()

epochs = pd.DataFrame(
    {"start": data["time"][:, 0], "stop": data["time"][:, 1], "label": ""}
)
metadata = {"channels": data["channel"], "thresh": data["threshold"]}
data = {"epochs": epochs, "metadata": metadata}

# np.save(file, data)


In [ ]:
class A:
    def __init__(self) -> None:
        pass


class B(A):
    def __init__(self) -> None:
        pass


a = B()

isinstance(a, B)


In [ ]:
import pandas as pd

a = pd.DataFrame({"a": [1, 2], "b": [3, 5]})
b = pd.DataFrame({"a": [7, 8], "b": [3, 5], "c": [3, 4]})

c = a.append(b)
np.__version__

In [ ]:
import subjects
from ModulesPath.core import Position, Track

sess = subjects.Sd().ratSday3[0]

pos = Position(
    time=sess.position.t,
    x=sess.position.x,
    y=sess.position.y,
    sampling_rate=int(sess.position.tracking_sRate),
)

int(pos.sampling_rate)

pos.linearize(period=sess.paradigm.maze1,sample_sec=3)
pos.linearize(period=sess.paradigm.maze2,sample_sec=3)

In [ ]:
import numpy as np
from probeinterface import Probe, ProbeGroup
from probeinterface.plotting import plot_probe_group, plot_probe

n = 24
positions = np.zeros((n, 2))
for i in range(n):
    x = i // 8
    y = i % 8
    positions[i] = x, y
positions *= 20
positions[8:16, 1] -= 10

probe_2d = Probe(ndim=2, si_units='um')
probe_2d.set_contacts(positions=positions, shapes='circle', shape_params={'radius': 5})
probe_2d.create_auto_shape(probe_type='tip')

In [ ]:
plot_probe(probe_2d)